In [1]:
# Import necessary packages
!pip install dmba
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import time

### Load required data from Evolving Hockey

In [2]:
# Read in both the standard and GAR csv files
standard = pd.read_csv("eh_standard_2324.csv")
gar = pd.read_csv("eh_gar_2324.csv")

# Convert csv files to dataframes
std_df = pd.DataFrame(standard)
gar_df = pd.DataFrame(gar)

In [3]:
# Merge the standard and GAR dataframes
fulldata = pd.merge(gar_df, std_df, on='API ID', how='outer')
fulldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824 entries, 0 to 823
Data columns (total 63 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player_x    824 non-null    object 
 1   EH_ID_x     824 non-null    object 
 2   API ID      824 non-null    int64  
 3   Season_x    824 non-null    object 
 4   Team_x      824 non-null    object 
 5   Position_x  824 non-null    object 
 6   Shoots_x    824 non-null    object 
 7   Birthday_x  824 non-null    object 
 8   Age_x       824 non-null    int64  
 9   Draft Yr_x  824 non-null    int64  
 10  Draft Rd_x  715 non-null    float64
 11  Draft Ov_x  715 non-null    float64
 12  GP_x        824 non-null    int64  
 13  TOI_All     824 non-null    float64
 14  EVO_GAR     824 non-null    float64
 15  EVD_GAR     824 non-null    float64
 16  PPO_GAR     824 non-null    float64
 17  SHD_GAR     824 non-null    float64
 18  Take_GAR    824 non-null    float64
 19  Draw_GAR    824 non-null    f

In [4]:
# Drop unnecessary columns
fulldata = fulldata.drop(columns=[
    'Shoots_x', 'Birthday_x', 'Draft Yr_x', 'Draft Rd_x', 'Draft Ov_x',
    'EVO_GAR', 'EVD_GAR', 'PPO_GAR','SHD_GAR', 'Take_GAR', 
    'Draw_GAR', 'Off_GAR', 'Def_GAR', 'Pens_GAR','Player_y', 
    'EH_ID_y', 'Season_y', 'Team_y','Position_y', 'Shoots_y', 
    'Birthday_y', 'Age_y', 'Draft Yr_y','Draft Rd_y', 'Draft Ov_y', 
    'GP_y','TOI','iSF', 'iFF', 'iCF', 'ixG', 'Sh%', 'FSh%', 
    'xFSh%', 'iBLK', 'GIVE','TAKE', 'iHF', 'iHA', 'iPENT2', 
    'iPEND2', 'iPENT5', 'iPEND5', 'iPEN±','FOW', 'FOL', 'FO±', 'Season_x'])

In [5]:
# Rename columns
fulldata = fulldata.rename(columns={
    'Player_x':'Player_Name', 
    'EH_ID_x':'Player_Name_Std', 
    'API ID':'Player_ID',
    'Team_x':'Team', 
    'Position_x':'Pos',
    'Age_x':'Age', 
    'GP_x':'GP', 
    'TOI_All':'TOI'})

In [6]:
# Convert TOI from total to per game
fulldata['TOI'] = fulldata['TOI'] / fulldata['GP']

In [7]:
# Preview the data and convert to csv
print(fulldata.head())
# (KEEP THE FOLLOWING AS A COMMENT SO IT IS NOT RUN EVERYTIME THE KERNEL RESTARTS)
# fulldata.to_csv('eh_fulldata_2324.csv', index=False)

   Player_Name Player_Name_Std  Player_ID     Team Pos  Age  GP        TOI  \
0   Ryan Suter      RYAN.SUTER    8470600      DAL   D   38  82  18.940244   
1  Jeff Carter     JEFF.CARTER    8470604      PIT   C   38  72  12.570833   
2  Zach Parise     ZACH.PARISE    8470610      COL   L   39  30  12.966667   
3  Brent Burns     BRENT.BURNS    8470613      CAR   D   38  82  21.501220   
4  Corey Perry     COREY.PERRY    8470621  CHI/EDM   R   38  54  13.381481   

    GAR  WAR  SPAR   G  A1  A2  Points  
0   5.1  0.8   1.6   2   7   8      17  
1  -7.6 -1.2  -2.4  11   2   2      15  
2  -1.2 -0.2  -0.4   5   2   3      10  
3  14.5  2.4   4.6  10  14  19      43  
4   6.9  1.1   2.2  12   6   4      22  


### Webscrape required data from PuckPedia

In [8]:
# Obtain the page source and create a beautiful soup object
url = 'https://puckpedia.com/players/search?s=160&r=1&ss=160'
driver = webdriver.Edge()
driver.get(url)
src = driver.page_source
soup = bs(src, 'html.parser')

In [9]:
#nextpg = driver.find_element(By.XPATH, '//*[@id="db_pagingwrap"]/div[2]/button[2]')

In [10]:
# Get a list of all player names, contract lengths, cap hits, start years, end years, and contract types
table_data = []


while True:
    #retrieve page source and parse with beutiful soup
    src = driver.page_source
    soup = bs(src, 'html.parser')
    
    #extract table rows
    rows = soup.find_all('tr')

    #iterate through each row and extract the column data
    for row in rows:
        columns = row.find_all('td')                                                              #identifies all td (column) elements in a single tr (row)
        row_data = [data.get_text(strip=True) for data in columns]                                #takes the text content from each column and makes a list of strings
        if row_data:                                                                              #if row data is not empty
            if len(row_data) > 1 and ',' in row_data[1]:                                          #if the second column has a player name like (LastName,FirstName)
                name_split = row_data[1].split(',')                                               #splits and reformats name in a list
                First_Name = name_split[1]
                Last_Name = name_split[0]                                 
                Contract_Length = row_data[13]
                Cap_Hit = row_data[15]
                Start_Year = row_data[24]
                Expiry_Year = row_data[27]
                Signing_Status = row_data[26]
                Expiry_Status = row_data[28]
                t = (First_Name, Last_Name, Contract_Length, Cap_Hit, Start_Year, Expiry_Year, Signing_Status, Expiry_Status)         #create a tuple with all of the necessary data
                if t not in table_data:                                                                                               #avoid duplicates
                    table_data.append(t)                                                                                              #append the tuples to table data list

    #click the next page button
    try:
        nextpg = driver.find_element(By.XPATH, '//*[@id="db_pagingwrap"]/div[2]/button[2]')                                                                                 
        if 'opacity-25' in nextpg.get_attribute('class'):                                         #check if the button is disabled
            break                                                                                 #exit loop if no next button
        nextpg.click()                                                                            #click next page button
        time.sleep(5)                                                                             #allow time for page to load
    except (NoSuchElementException, StaleElementReferenceException):
        break                                                                                     #exit the loop if next page cannot be found or is stale


playerdata_df = pd.DataFrame(table_data, columns=['First_Name', 'Last_Name', 'Contract_Length', 'Cap_Hit', 'Start_Year', 'Expiry_Year', 'Signing_Status', 'Expiry_Status'])

In [11]:
# Convert playerdata_df to a csv file 
# (KEEP THE FOLLOWING AS A COMMENT SO IT IS NOT RUN EVERYTIME THE KERNEL RESTARTS)
# playerdata_df.to_csv('pp_contract_2324.csv', index=False)

### Prepare dataframes for merging

In [12]:
# Read in data
contracts = pd.read_csv('pp_contract_2324.csv')
contracts_df = pd.DataFrame(contracts)


fulldata = pd.read_csv('eh_fulldata_2324.csv')
fulldata_df = pd.DataFrame(fulldata)

In [13]:
# Some players have special characters in their names -> Juraj Slafkovsky, Janis Jerome Moser, Alexis Lafreniere, Aatu Raty, Tim Stutzle.  Fix their names in both contracts and fulldata so they match
contracts_df.loc[contracts_df['First_Name'] == 'Janis Jérôme', 'First_Name'] = 'Janis'
contracts_df.loc[contracts_df['Last_Name'] == 'Slafkovský', 'Last_Name'] = 'Slafkovsky'
contracts_df.loc[contracts_df['Last_Name'] == 'Räty', 'Last_Name'] = 'Raty'

fulldata_df.loc[fulldata_df['Player_Name'] == 'Alexis Lafrenière', 'Player_Name'] = 'Alexis Lafreniere'
fulldata_df.loc[fulldata_df['Player_Name'] == 'Tim Stützle', 'Player_Name'] = 'Tim Stutzle'

fulldata_df.loc[fulldata_df['Player_Name_Std'] == 'ALEXIS.LAFRENIÈRE', 'Player_Name_Std'] = 'ALEXIS.LAFRENIERE'
fulldata_df.loc[fulldata_df['Player_Name_Std'] == 'TIM.STÜTZLE', 'Player_Name_Std'] = 'TIM.STUTZLE'

In [14]:
# Create a new column in contracts_df (Player_Name_Std) which you can merge the two dataframes on
Player_Name_Stdlist = []
for _, x in contracts_df.iterrows():
    fncaps = x['First_Name'].upper()
    lncaps = x['Last_Name'].upper()
    std_name = f"{fncaps}.{lncaps}"
    Player_Name_Stdlist.append(std_name)

contracts_df['Player_Name_Std'] = Player_Name_Stdlist

In [15]:
# Some names do not match up in both of the data frames (ex: Alex vs Alexander). Format the mismatch names from contracts_df in the same manner as fulldata_df
before = ['ALEXANDER.ALEXEYEV', 'ALEXANDER.BARABANOV', 'ALEXANDER.HOLTZ', 'ALEXANDER.KERFOOT', 'ALEXANDER.ROMANOV', 
              'ALEXANDER.WENNBERG', 'ALEXANDRE.TEXIER', 'ARTYOM.ZUB', 'CALLAN.FOOTE', 'CAMERON.YORK', 
              'CHRISTOPHER.TANEV', 'EMIL.LILLEBERG', 'EVGENII.DADONOV', 'JOHN-JASON.PETERKA', 'JOSEPH.VELENO', 
              'JOSHUA.NORRIS', 'MATTHEW.BENIERS', 'MITCHELL.MARNER', 'NICKLAUS.PERBIX', 'PAT.MAROON', 
              'ZACHARY.JONES', 'ERIK.GUSTAFSSON', 'ALEXANDER.NYLANDER', 'ALEXANDRE.CARRIER', 'COLIN.WHITE', 'RYAN.JOHNSON', 'YEGOR.ZAMULA']
after = ['ALEX.ALEXEYEV', 'ALEX.BARABANOV', 'ALEX.HOLTZ', 'ALEX.KERFOOT', 'ALEX.ROMANOV', 
             'ALEX.WENNBERG', 'ALEX.TEXIER', 'ARTEM.ZUB', 'CAL.FOOTE', 'CAM.YORK', 
             'CHRIS.TANEV', 'EMIL.MARTINSEN LILLEBERG', 'EVGENY.DADONOV', 'JJ.PETERKA', 'JOE.VELENO', 
             'JOSH.NORRIS', 'MATTY.BENIERS', 'MITCH.MARNER', 'NICK.PERBIX', 'PATRICK.MAROON', 
             'ZAC.JONES', 'ERIK.GUSTAFSSON2', 'ALEX.NYLANDER', 'ALEX.CARRIER', 'COLIN.WHITE2', 'RYAN.JOHNSON2', 'EGOR.ZAMULA']

for before, after in zip(before, after):
    contracts_df.loc[contracts_df['Player_Name_Std'] == before, 'Player_Name_Std'] = after

In [16]:
# PLAYER SEARCH TO CHECK CHANGES
contracts_df.loc[contracts_df['Last_Name'] == 'Holtz']

,First_Name,Last_Name,Contract_Length,Cap_Hit,Start_Year,Expiry_Year,Signing_Status,Expiry_Status,Player_Name_Std
551,Alexander,Holtz,3,"$894,167",2021-22,2024-25,+ARB,RFA+ARBRFA,ALEX.HOLTZ


In [17]:
# Convert the Cap_Hit column to an integer value (in millions)
for x in contracts_df['Cap_Hit']:
    contracts_df['Cap_Hit'] = contracts_df['Cap_Hit'].replace(x, x.replace(',', '').replace('$', ''))

contracts_df['Cap_Hit'] = contracts_df['Cap_Hit'].astype('int')

In [18]:
# Convert Start_Year and Expiry_Year columns to an integer value
contracts_df['Start_Year'] = contracts_df['Start_Year'].str.split('-', expand=True)[0]
contracts_df['Expiry_Year'] = contracts_df['Expiry_Year'].str.split('-', expand=True)[0]

contracts_df['Start_Year'] = contracts_df['Start_Year'].astype('int')
contracts_df['Expiry_Year'] = contracts_df['Expiry_Year'].astype('int')

In [19]:
# Strip Signing_Status and Expiry_Status of unnecessary text
contracts_df['Signing_Status'] = contracts_df['Signing_Status'].str.slice(0, 3)
contracts_df['Expiry_Status'] = contracts_df['Expiry_Status'].str.slice(0, 3)

In [20]:
# Replace entries containing '+AR' for Signing_Status and Expiry_Status with 'ELC'
for x in contracts_df['Signing_Status']:
    if x == '+AR':
        contracts_df.loc[contracts_df['Signing_Status'] == x, 'Signing_Status'] = 'ELC'

for x in contracts_df['Expiry_Status']:
    if x == '+AR':
        contracts_df.loc[contracts_df['Expiry_Status'] == x, 'Expiry_Status'] = 'ELC'

In [21]:
# Get dummy variables for both the Signing_Status and Expiry_Status columns (0=UFA, 1=RFA, 2=ELC)
contracts_df = pd.get_dummies(contracts_df, columns=['Signing_Status'], dtype='int')
contracts_df = pd.get_dummies(contracts_df, columns=['Expiry_Status'], dtype='int')

In [22]:
# Drop unnecessary dummy columns and rename
contracts_df = contracts_df.drop(columns=['Expiry_Status_RFA', 'Signing_Status_ELC'])

contracts_df = contracts_df.rename(columns={
    'Expiry_Status_UFA':'Expiry_Status'})

### Merge the two dataframes

In [23]:
final_df = pd.merge(fulldata_df, contracts_df, on='Player_Name_Std', how='outer')

In [24]:
# Remove rows with null values --> either missing contract info or statistical info (did not play due to injury, minor league contract, etc.)
final_df = final_df.dropna(subset=['Player_Name', 'First_Name'])  

# Using subset 'Player_Name' and 'First_Name' takes out rows that did not merge properly, but leaves rows that did merge properly and may be missing a statistic

In [25]:
# Export merged dataframe
# (KEEP THE FOLLOWING AS A COMMENT SO IT IS NOT RUN EVERYTIME THE KERNEL RESTARTS)
final_df.to_csv('dw_finaldata_2324.csv', index=False)

### Prepare the new dataframe

In [26]:
# Read in the new csv file
final_df = pd.read_csv('dw_finaldata_2324.csv')

In [27]:
# Convert necessary columns from float to integer
final_df[['Player_ID', 'Age', 'GP', 'G', 'A1', 'A2', 'Points', 
          'Contract_Length', 'Cap_Hit', 'Start_Year', 'Expiry_Year', 
          'Signing_Status_RFA', 'Signing_Status_UFA', 'Expiry_Status']] = final_df[['Player_ID', 'Age', 'GP', 'G', 'A1', 'A2', 'Points', 
                                                                                    'Contract_Length', 'Cap_Hit', 'Start_Year', 'Expiry_Year', 
                                                                                    'Signing_Status_RFA', 'Signing_Status_UFA', 'Expiry_Status']].astype('int')

In [28]:
# Scale Cap_Hit and GAR, WAR, and SPAR
scaler = StandardScaler()
final_df[['Cap_Hit', 'GAR', 'WAR', 'SPAR']] = scaler.fit_transform(final_df[['Cap_Hit', 'GAR', 'WAR', 'SPAR']])

# Create new columns to allow for data analysis (Surplus_Value and Evaluation)
final_df['Surplus_Value'] = final_df['GAR'] - final_df['Cap_Hit']
final_df['Evaluation'] = final_df['Surplus_Value'].apply(lambda x: 1 if x > 0 else 0)

In [29]:
# Check how many steals/overpays there are. Evaluation: 1=Steal, 0=Overpay
print(final_df['Evaluation'].value_counts())   
final_df[['Player_Name_Std', 'Evaluation', 'Surplus_Value']].sort_values(by='Surplus_Value', ascending=False)

Evaluation
1    368
0    316
Name: count, dtype: int64


,Player_Name_Std,Evaluation,Surplus_Value
587,SETH.JARVIS,1,3.711920
78,BRANDON.HAGEL,1,3.075959
610,THOMAS.HARLEY,1,3.049798
677,ZACH.HYMAN,1,2.680309
32,ALEX.VLASIC,1,2.500308
...,...,...,...
477,NICKLAS.BACKSTROM,0,-2.992906
315,JONATHAN.HUBERDEAU,0,-3.064902
108,CAM.FOWLER,0,-3.070393
251,JACOB.TROUBA,0,-3.086242


In [31]:
final_df.loc[final_df['Team'] == 'BOS'].sort_values(by='Surplus_Value', ascending=False)

,Player_Name,Player_Name_Std,Player_ID,Team,Pos,Age,GP,TOI,GAR,WAR,...,Last_Name,Contract_Length,Cap_Hit,Start_Year,Expiry_Year,Signing_Status_RFA,Signing_Status_UFA,Expiry_Status,Surplus_Value,Evaluation
587,Seth Jarvis,SETH.JARVIS,8482093,CAR,C,21,81,18.750617,2.833184,2.873848,...,Jarvis,3,-0.878737,2020,2023,0,0,0,3.711920,1
78,Brandon Hagel,BRANDON.HAGEL,8479542,T.B,L,25,82,19.256098,2.417716,2.437353,...,Hagel,3,-0.658242,2021,2023,1,0,0,3.075959,1
610,Thomas Harley,THOMAS.HARLEY,8481581,DAL,D,22,79,21.016456,2.159840,2.175456,...,Harley,3,-0.889958,2019,2023,0,0,0,3.049798,1
677,Zach Hyman,ZACH.HYMAN,8475786,EDM,L,31,80,19.500000,3.477875,3.484942,...,Hyman,7,0.797566,2021,2027,0,1,1,2.680309,1
32,Alex Vlasic,ALEX.VLASIC,8481568,CHI,D,22,76,21.478947,1.629761,1.651662,...,Vlasic,3,-0.870548,2021,2023,0,0,0,2.500308,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,Nicklas Backstrom,NICKLAS.BACKSTROM,8473563,WSH,C,35,8,14.562500,-0.848717,-0.880010,...,Backstrom,5,2.144188,2020,2024,0,1,1,-2.992906,0
315,Jonathan Huberdeau,JONATHAN.HUBERDEAU,8476456,CGY,C,30,81,17.422222,-0.447576,-0.443515,...,Huberdeau,8,2.617326,2023,2030,0,1,1,-3.064902,0
108,Cam Fowler,CAM.FOWLER,8475764,ANA,D,31,81,24.423457,-1.908876,-1.927598,...,Fowler,8,1.161518,2018,2025,0,1,1,-3.070393,0
251,Jacob Trouba,JACOB.TROUBA,8476885,NYR,D,29,69,21.246377,-1.378796,-1.403804,...,Trouba,7,1.707446,2019,2025,1,0,1,-3.086242,0
